In [1]:
import pandas as pd
import urllib3
import requests
import certifi
from bs4 import BeautifulSoup
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
dt_rwt = pd.read_excel('tar_work.xlsm',sheet_name ='rwst')
dt_rwt.head()

,Станции отправления,Регион отправления,Коды,Регион получения,Станции получения,Коды1,Груз,Ж/Д Тариф с охраной и ндс,Время в пути,Расстояние,СПИМЕКС,Надбавка за СНГ
0,аксарайская ii,АСТРАХАНСКАЯ ОБЛАСТЬ,617007,алтайский край,усть-тальменская,840908,АБ,NaN,NaN,NaN,NaN,NaN
1,аллагуват,РЕСПУБЛИКА БАШКОРТОСТАН,652808,алтайский край,усть-тальменская,840908,АБ,NaN,NaN,NaN,NaN,NaN
2,ачинск ii,КРАСНОЯРСКИЙ КРАЙ,883809,алтайский край,усть-тальменская,840908,АБ,NaN,NaN,NaN,NaN,NaN
3,батарейная,ИРКУТСКАЯ ОБЛАСТЬ,932601,алтайский край,усть-тальменская,840908,АБ,NaN,NaN,NaN,NaN,NaN
4,бензин,РЕСПУБЛИКА БАШКОРТОСТАН,654909,алтайский край,усть-тальменская,840908,АБ,NaN,NaN,NaN,NaN,NaN


dt_rwt=dt_rwt.loc[dt_rwt['Груз']=='ДТ']

In [3]:
dt_rwt.iloc[:,7] = None
dt_rwt.iloc[:,8] = None
dt_rwt.iloc[:,9] = None

In [10]:
# iteration variable
k=0
start = time.time()

rangestart = 0
rangeend = dt_rwt.shape[0]
for i in range(rangestart, rangeend):

    ## get variables
    departure_station = str(dt_rwt.iloc[i,2])
    if(len(departure_station)<6):
        departure_station = '0'*(6-len(departure_station)) + departure_station
    
    receiving_station = str(dt_rwt.iloc[i,5])
    if(len(receiving_station)<6):
        receiving_station = '0'*(6-len(receiving_station)) + receiving_station
    
    np_type = dt_rwt.iloc[i,6]
    
    ## get link
    if (np_type=='АБ'):
        text_url = 'http://spimex.com/tar/prod2009/tar_raschet_only.php?tarID=12&pr_sprav=5&st1='+departure_station+'&st2='+receiving_station+'&kgr=21102&gp=52&tip=3ves=52&sv=2&nv=1&osi=4&nvohr=1&nprov=0&period=last&podstanovka=0&valid=1&view=tab'
        try:
            spimex_table = requests.get( text_url, verify=False)
            soup = BeautifulSoup( spimex_table.text, 'html.parser')
            soup = soup.find_all('td')

            ## distance
            dt_rwt.iloc[i,9] = float( soup[2].text.replace(' км.',''))
            ## time
            dt_rwt.iloc[i,8] = float(soup[4].text.replace(' сут.',''))
            ## tarif
            dt_rwt.iloc[i,7] = float(soup[-2].text)/52
        except:
            ## distance
            dt_rwt.iloc[i,9] = None
            ## time
            dt_rwt.iloc[i,8] = None
            ## tarif
            dt_rwt.iloc[i,7] = None
    else:
        text_url = 'http://spimex.com/tar/prod2009/tar_raschet_only.php?tarID=12&pr_sprav=5&st1='+departure_station+'&st2='+receiving_station+'&kgr=21400&gp=55&tip=3ves=52&sv=2&nv=1&osi=4&nvohr=1&nprov=0&period=last&podstanovka=0&valid=1&view=tab'
        try:
            spimex_table = requests.get( text_url, verify=False)
            soup = BeautifulSoup( spimex_table.text, 'html.parser')
            soup = soup.find_all('td')

            ## distance
            dt_rwt.iloc[i,9] = float( soup[2].text.replace(' км.',''))
            ## time
            dt_rwt.iloc[i,8] = float(soup[4].text.replace(' сут.',''))
            ## tarif
            dt_rwt.iloc[i,7] = float(soup[-2].text)/55
        except:
            ## distance
            dt_rwt.iloc[i,9] = None
            ## time
            dt_rwt.iloc[i,8] = None
            ## tarif
            dt_rwt.iloc[i,7] = None        
            
    if (k%100 == 0):
        time.sleep(5)
    if ((i%5000 == 0) or (i == (rangeend-1))) and not (i == 0):
        dt_rwt.to_excel('dt_rwt_dt_end2.xlsx', index = False)
        print('скачал ' + str(i))
        time_i = (time.time() - start)/3600
        print('time: '+ str(int(time_i)) + ' hours ' + str(round((time_i - int(time_i)) * 60 , ndigits=1)) + ' minutes')
        start = time.time()

скачал 111
time: 0 hours 10 minutes
